In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf

pd.set_option("display.max_rows", 200)
pd.set_option("display.width", 140)

In [ ]:
df = pd.read_csv("abstracts_temp_9000.csv")

df.head(3), df.shape, df.dtypes

In [ ]:
print("Before:", df.shape)
df = df[~df['year'].isin([2024, 2025])]
print("After:", df.shape)

In [ ]:
print(df['year'].value_counts().sort_index())

In [ ]:
# Basic checks
assert {"id","year","authors_count","source"}.issubset(df.columns), "Missing expected columns."
df = df.drop_duplicates("id").copy()

# Keep relevant window (we’ll still run an all-years ‘first look’ exactly like yours)
df_1825 = df[(df["year"] >= 2018) & (df["year"] <= 2025)].copy()
df_1825.shape

### Pandemic-free Split

In [ ]:
pre_ai = df[df["year"] < 2021]
post_ai = df[df["year"] >= 2021]

print("🧠 AI PARADOX — all years")
print(f"Pre-2021:  {len(pre_ai):,} papers, {pre_ai['authors_count'].mean():.1f} avg authors")
print(f"Post-2021: {len(post_ai):,} papers, {post_ai['authors_count'].mean():.1f} avg authors")

### Pandemic-free Contrast (skip 2020–2021, and use 2018–19 vs 2022–23)

In [ ]:
pre = df_1825[df_1825["year"].between(2018, 2019)]
post = df_1825[df_1825["year"].between(2022, 2023)]
print("🧠 AI PARADOX — pandemic-free window")
print(f"Pre (2018–19):  {len(pre):,} papers, {pre['authors_count'].mean():.1f} avg authors")
print(f"Post (2022–23): {len(post):,} papers, {post['authors_count'].mean():.1f} avg authors")

### Field Tagging

In [ ]:
def map_field_from_source(s):
    if not isinstance(s,str): return "Other"
    v = s.lower()
    # Medical
    medical_keys = ["jama","lancet","nejm","bmj","clinic","clinical","medicine","oncology","cardio","nursing","public health","epidemiol","psychiatry","pediatr","radiol"]
    if any(k in v for k in medical_keys): return "Medical"
    # STEM (non-med)
    stem_keys = ["physics","chemistry","chemical","biol","materials","engineering","computer","information","neuroscience","math","mathematics","statistics","ai","robotics","electrical","optics","geology","earth","astronomy","environmental science"]
    if any(k in v for k in stem_keys): return "STEM"
    # Social
    social_keys = ["economics","sociology","psychology","education","political","management","communication","marketing","journalism","public policy","development studies","anthropology","demography","geography","social","policy"]
    if any(k in v for k in social_keys): return "Social"
    # Humanities
    hum_keys = ["history","philosophy","literature","linguistics","arts","humanities","cultural","theology","religion","poetry"]
    if any(k in v for k in hum_keys): return "Humanities"
    return "Other"

df_1825["field_group"] = df_1825["source"].apply(map_field_from_source)
df_1825["post2021"] = (df_1825["year"] >= 2022).astype(int)  # post-LLM diffusion ≈ 2022+

### Despcriptives

In [ ]:
# Overall by year
overall = (
    df_1825
    .groupby("year")["authors_count"]
    .agg(n="count", mean="mean", median="median", p90=lambda x: np.percentile(x,90))
    .round(2)
)
display(overall)
overall.to_csv("tab_overall_year.csv")

# Field × year panel
desc = (
    df_1825
    .groupby(["field_group","year"])["authors_count"]
    .agg(n="count", mean="mean", median="median", p90=lambda x: np.percentile(x, 90))
    .reset_index()
    .sort_values(["field_group","year"])
    .round(2)
)
display(desc.head(20))
desc.to_csv("tab_field_year.csv", index=False)

In [ ]:
data = {
    "year": [2018,2019,2020,2021,2022,2023],
    "mean": [12.3,11.9,14.1,17.4,20.8,21.6],
    "p90": [26,24,33,42,49,65]
}
df_summary = pd.DataFrame(data)

plt.figure(figsize=(8,5))
plt.plot(df_summary["year"], df_summary["mean"], marker="o", label="Mean authors")
plt.plot(df_summary["year"], df_summary["p90"], marker="s", label="90th percentile")
plt.xlabel("Year")
plt.ylabel("Authors per paper")
plt.title("Mean and 90th Percentile Author Counts, 2018–2023")
plt.grid(True, linewidth=0.3)
plt.legend()
plt.tight_layout()
plt.show()

### Pandemic-free within-field deltas (2018–19 vs 2022–23) + bootstrap CIs

In [ ]:
def bootstrap_ci_diff(a, b, B=2000, alpha=0.05, seed=42):
    rng = np.random.default_rng(seed)
    diffs = []
    for _ in range(B):
        da = rng.choice(a, size=len(a), replace=True).mean()
        db = rng.choice(b, size=len(b), replace=True).mean()
        diffs.append(db - da)
    low = np.percentile(diffs, 100*alpha/2)
    high = np.percentile(diffs, 100*(1-alpha/2))
    return np.mean(b)-np.mean(a), low, high

rows = []
for fg, sub in df_1825.groupby("field_group"):
    a = sub.loc[sub["year"].between(2018,2019),"authors_count"].values
    b = sub.loc[sub["year"].between(2022,2023),"authors_count"].values
    if len(a)>=20 and len(b)>=20:
        d, lo, hi = bootstrap_ci_diff(a,b)
        rows.append((fg, len(a), len(b), np.mean(a), np.mean(b), d, lo, hi))
contrast = pd.DataFrame(rows, columns=["field_group","n_pre","n_post","pre_mean","post_mean","delta","ci_low","ci_high"]).round(2)
display(contrast.sort_values("delta"))
contrast.to_csv("tab_delta_pandemic_free.csv", index=False)

### Regression: Structural break (post-2021) with year fixed effects

In [ ]:
m_df = df_1825.dropna(subset=["authors_count","year"]).copy()

# OLS with year FE and field interactions
ols = smf.ols("authors_count ~ post2021 * C(field_group) + C(year)", data=m_df).fit(cov_type="HC3")
coefs = ols.params[ols.params.index.str.contains("post2021")].round(3).sort_index()
pvals = ols.pvalues[coefs.index].round(4)
display(pd.DataFrame({"coef": coefs, "pval": pvals}))

### Regression: Robustness trio (raw, log1p, capped at 50)

In [ ]:
def run_ols_spec(y_col, data):
    f = f"{y_col} ~ post2021 * C(field_group) + C(year)"
    res = smf.ols(f, data=data).fit(cov_type="HC3")
    keep = res.params.index.str.contains("post2021")
    out = pd.DataFrame({
        "term": res.params.index[keep],
        "coef": res.params[keep].round(3),
        "pval": res.pvalues[keep].round(4)
    }).sort_values("term")
    return out

robust_df = m_df.copy()
robust_df["authors_log1p"] = np.log1p(robust_df["authors_count"])
robust_df["authors_cap50"] = robust_df["authors_count"].clip(upper=50)

specA = run_ols_spec("authors_count", robust_df)
specB = run_ols_spec("authors_log1p", robust_df)
specC = run_ols_spec("authors_cap50", robust_df)

print("Spec A — raw authors"); display(specA)
print("Spec B — log1p"); display(specB)
print("Spec C — cap=50"); display(specC)

specA.to_csv("tab_ols_specA_raw.csv", index=False)
specB.to_csv("tab_ols_specB_log1p.csv", index=False)
specC.to_csv("tab_ols_specC_cap50.csv", index=False)

### Minimal visuals (field-wise trend with bootstrapped 95% CI)

In [ ]:
def bootstrap_mean_ci(x, B=800, alpha=0.05, seed=42):
    rng = np.random.default_rng(seed)
    x = np.asarray(x, dtype=float)
    boots = [rng.choice(x, size=len(x), replace=True).mean() for _ in range(B)]
    low = np.percentile(boots, 100*alpha/2)
    high = np.percentile(boots, 100*(1-alpha/2))
    return float(np.mean(x)), float(low), float(high)

# summary table
plot_rows = []
for (fg, yr), sub in df_1825.groupby(["field_group", "year"]):
    if len(sub) >= 5:
        m, lo, hi = bootstrap_mean_ci(sub["authors_count"].values)
        plot_rows.append((fg, int(yr), m, lo, hi, int(len(sub))))
plot_df = pd.DataFrame(
    plot_rows,
    columns=["field_group","year","mean","ci_low","ci_high","n"]
).sort_values(["field_group","year"])

# Single figure with all fields
fig, ax = plt.subplots(figsize=(9,5))

for fg, sub in plot_df.groupby("field_group"):
    sub = sub.sort_values("year")
    ax.plot(sub["year"], sub["mean"], marker="o", label=str(fg))
    ax.fill_between(sub["year"], sub["ci_low"], sub["ci_high"], alpha=0.15)

overall = (
    df_1825.groupby("year")["authors_count"]
    .apply(lambda x: bootstrap_mean_ci(x.values)[0])
    .reset_index(name="mean")
    .sort_values("year")
)
ax.plot(overall["year"], overall["mean"], linestyle="--", label="Overall")

ax.axhline(0, color='blue', lw=0.3) 

ax.set_title("Mean Authors per Paper (2018–2023) by Field")
ax.set_xlabel("Year")
ax.set_ylabel("Mean authors")
ax.grid(True, linewidth=0.3)
ax.legend(title="Field", bbox_to_anchor=(1.02, 1), loc="upper left")
fig.tight_layout()
plt.show()

### Save key tables

In [ ]:
overall.to_csv("tab_overall_year.csv")
desc.to_csv("tab_field_year.csv", index=False)
contrast.to_csv("tab_delta_pandemic_free.csv", index=False)
specA.to_csv("tab_ols_specA_raw.csv", index=False)
specB.to_csv("tab_ols_specB_log1p.csv", index=False)
specC.to_csv("tab_ols_specC_cap50.csv", index=False)

### AI Paradox 

In [ ]:
pre_ai = df[df['year'] < 2021]
post_ai = df[df['year'] >= 2021]

print("AI PARADOX FIRST LOOK:")
print(f"Pre-2021:  {len(pre_ai):,} papers, {pre_ai['authors_count'].mean():.1f} avg authors")
print(f"Post-2021: {len(post_ai):,} papers, {post_ai['authors_count'].mean():.1f} avg authors")

### Top Sources Comparison

In [ ]:
# Are post-2021 papers from different types of sources?
post_ai_sources = df[df['year'] >= 2021]['source'].value_counts().head(10)
pre_ai_sources = df[df['year'] < 2021]['source'].value_counts().head(10)

print("TOP SOURCES COMPARISON:")
print("Post-2021 (High-author era):")
print(post_ai_sources)
print("\nPre-2021 (Baseline era):")
print(pre_ai_sources)

# Check author count distribution
print(f"\n AUTHOR COUNT DISTRIBUTION:")
print(f"Pre-2021  max authors: {pre_ai['authors_count'].max()}")
print(f"Post-2021 max authors: {post_ai['authors_count'].max()}")

### Authorship Inflation Within Top Journals 

In [ ]:
common_top_sources = ['New England Journal of Medicine', 'Science', 'Nature Communications', 'Nucleic Acids Research']

print("Authorship Inflation Within Top Journals:")
for journal in common_top_sources:
    journal_data = df[df['source'] == journal]
    pre = journal_data[journal_data['year'] < 2021]['authors_count'].mean()
    post = journal_data[journal_data['year'] >= 2021]['authors_count'].mean()
    if not (pd.isna(pre) or pd.isna(post)):
        change = ((post - pre) / pre) * 100
        print(f"{journal}: {pre:.1f} → {post:.1f} authors ({change:+.1f}%)")

### Investigating whether Nature Comm is outlier

In [ ]:
nature_comm = df[df['source'] == 'Nature Communications']
print("NATURE COMMUNICATIONS DEEP DIVE:")
print(f"Total papers: {len(nature_comm)}")
print(f"Year distribution:")
print(nature_comm['year'].value_counts().sort_index())
print(f"Author count range: {nature_comm['authors_count'].min()} - {nature_comm['authors_count'].max()}")

### Year-Distribution Across Elite Journals

In [ ]:
# Compare year distributions across top journals
print("YEAR DISTRIBUTION ACROSS ELITE JOURNALS:")
top_journals = ['New England Journal of Medicine', 'Science', 'Nature Communications', 'Nucleic Acids Research']

for journal in top_journals:
    j_data = df[df['source'] == journal]
    post_2021_ratio = len(j_data[j_data['year'] >= 2021]) / len(j_data)
    print(f"{journal}: {post_2021_ratio:.1%} post-2021 papers")

# Quick reality check - are we seeing publication lag?
print(f"\n OVERALL POST-2021 COVERAGE: {len(post_ai)/len(df):.1%}")

### Weighted Authorship Analysis 

In [ ]:
# Weighted analysis to account for temporal distribution
print("WEIGHTED AUTHORSHIP ANALYSIS:")

# Calculate field-specific trends with awareness of sampling realities
for journal in top_journals:
    j_data = df[df['source'] == journal]
    if len(j_data[j_data['year'] >= 2021]) > 10:  # Minimum sample threshold
        pre = j_data[j_data['year'] < 2021]['authors_count'].mean()
        post = j_data[j_data['year'] >= 2021]['authors_count'].mean()
        change = ((post - pre) / pre) * 100
        sample_size = len(j_data[j_data['year'] >= 2021])
        print(f"{journal}: {pre:.1f}→{post:.1f} ({change:+.1f}%) [n={sample_size} post-2021]")

### Visual - Post-2021 Authorship Inflation by Journal

In [ ]:
import matplotlib.pyplot as plt

# Simple bar chart of key finding
journals = ['Nucleic Acids\nResearch', 'NEJM', 'Science', 'Nature\nCommunications']
changes = [40.2, 20.6, 15.4, -8.5]

plt.figure(figsize=(10, 6))
bars = plt.bar(journals, changes, color=['red', 'orange', 'blue', 'green'])
plt.ylabel('Authorship Change (%)')
plt.title('Post-2021 Authorship Inflation by Journal')
plt.axhline(y=0, color='black', linestyle='-', alpha=0.3)

# Adding value labels on bars
for bar, value in zip(bars, changes):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1, 
             f'{value:+.1f}%', ha='center', va='bottom')

plt.tight_layout()
plt.show()

### Paper Characteristics by Journal

In [ ]:
print("PAPER CHARACTERISTICS BY JOURNAL:")
for journal in top_journals:
    j_data = df[df['source'] == journal]
    print(f"\n{journal}:")
    print(f"  Avg authors: {j_data['authors_count'].mean():.1f}")
    print(f"  Max authors: {j_data['authors_count'].max()}")
    print(f"  Papers with 10+ authors: {(j_data['authors_count'] >= 10).mean():.1%}")

### The Collaboration Spectrum 

In [ ]:
# The percentage of mega-teams tells the real story
print("THE COLLABORATION SPECTRUM:")
for journal in top_journals:
    j_data = df[df['source'] == journal]
    mega_team = (j_data['authors_count'] >= 20).mean() * 100
    hyper_team = (j_data['authors_count'] >= 50).mean() * 100
    print(f"{journal}:")
    print(f"  20+ authors: {mega_team:.1f}%")
    print(f"  50+ authors: {hyper_team:.1f}%")

### Four Collaboration Regimes in Elite Science

In [ ]:
# Collaboration regime plot

journals = ['NEJM', 'Science', 'NAR', 'Nature Comm']
small_team = [37.5, 67.6, 71.1, 89.4]  # <20 authors
mega_team = [62.5, 32.4, 28.9, 10.6]   # 20+ authors
hyper_team = [6.5, 8.1, 8.2, 2.6]      # 50+ authors

x = np.arange(len(journals))
width = 0.25

fig, ax = plt.subplots(figsize=(12, 8))
rects1 = ax.bar(x - width, small_team, width, label='<20 Authors', color='lightblue')
rects2 = ax.bar(x, mega_team, width, label='20+ Authors', color='orange')  
rects3 = ax.bar(x + width, hyper_team, width, label='50+ Authors', color='red')

ax.set_ylabel('Percentage of Papers', fontsize = 12)
ax.set_title('The Four Regimes of Scientific Collaboration', fontsize = 13)
ax.set_xticks(x)
ax.set_xticklabels(journals, fontsize = 12)
ax.legend()
ax.grid()

plt.tight_layout()
plt.show()